In [25]:
# ==============================================================================
# CÉLULA 1: IMPORTAÇÃO DE BIBLIOTECAS E CONFIGURAÇÃO
# ==============================================================================
import pandas as pd
import numpy as np
import io
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
import warnings
import kagglehub
import joblib

warnings.filterwarnings('ignore')

print("Bibliotecas importadas com sucesso.")


Bibliotecas importadas com sucesso.


In [26]:

# ==============================================================================
# CÉLULA 2: CARREGAMENTO E PREPARAÇÃO DOS DADOS (COM AMBOS OS CSVs)
# ==============================================================================

try:
    path = kagglehub.dataset_download("piterfm/massive-missile-attacks-on-ukraine")

    print("Path to dataset files:", path)

    attacks_df = pd.read_csv(f"{path}/missile_attacks_daily.csv")
    weapons_df= pd.read_csv(f"{path}/missiles_and_uav.csv")
    print("Arquivo 'missile_attacks_daily.csv' carregado com sucesso.")
except FileNotFoundError:
    print("Erro: Verifique se o arquivo 'missile_attacks_daily.csv' está no diretório correto.")
    exit()



Path to dataset files: /home/eznolin/.cache/kagglehub/datasets/piterfm/massive-missile-attacks-on-ukraine/versions/136
Arquivo 'missile_attacks_daily.csv' carregado com sucesso.


In [27]:
# Merge dos dataframes
df = pd.merge(attacks_df, weapons_df[['model', 'category', 'guidance_system']], on='model', how='left')

# Limpeza e processamento
df['event_date'] = pd.to_datetime(df['time_start'], format='mixed', errors='coerce')
df['year'] = df['event_date'].dt.year
numeric_cols = ['launched', 'destroyed']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
df['destroyed'] = df.apply(lambda row: min(row['launched'], row['destroyed']), axis=1)

# A 'category' agora é a nossa classificação principal e mais precisa.
# df['category'] = df['category'].fillna('Unknown')
print("\nDados de ataques enriquecidos com detalhes das armas. Amostra:")
print(df[['event_date', 'model', 'category', 'launched', 'destroyed', 'guidance_system']].head())


Dados de ataques enriquecidos com detalhes das armas. Amostra:
           event_date             model                category  launched  \
0 2025-06-21 21:00:00    Shahed-136/131                     UAV      47.0   
1 2025-06-21 21:00:00  Iskander-M/KN-23       ballistic missile       2.0   
2 2025-06-21 21:00:00             C-300  surface-to-air missile       1.0   
3 2025-06-20 20:00:00    Shahed-136/131                     UAV     272.0   
4 2025-06-21 02:40:00            Kalibr          cruise missile       4.0   

   destroyed guidance_system  
0       18.0             NaN  
1        0.0             NaN  
2        0.0             NaN  
3      140.0             NaN  
4        3.0             NaN  


In [28]:
# ==============================================================================
# CÉLULA 2.5: CÁLCULO DO P(TRACK) PONDERADO
# ==============================================================================
print("\n--- CALCULANDO P(TRACK) MÉDIO PONDERADO COM BASE NO INVENTÁRIO ---")

# Dados extraídos da imagem do inventário
inventory_data = {
    'system': ['Patriot PAC-3', 'SAMP/T', 'IRIS-T SLM', 'NASAMS', 'S-300', 'Buk-M1', 'Gepard'],
    'quantity': [3, 1, 13, 15, 100, 60, 52],
    'p_track_tier': [0.97, 0.96, 0.95, 0.95, 0.90, 0.90, 0.85]
}
inventory_df = pd.DataFrame(inventory_data)

# Calcular a média ponderada
total_quantity = inventory_df['quantity'].sum()
weighted_sum = (inventory_df['quantity'] * inventory_df['p_track_tier']).sum()
weighted_p_track = weighted_sum / total_quantity

# ATUALIZANDO A VARIÁVEL P_TRACK GLOBALMENTE
P_TRACK = round(weighted_p_track, 3) # Arredondando para 3 casas decimais

print(f"O P(track) médio ponderado calculado é: {P_TRACK}")
print("Este valor será usado em todos os cálculos subsequentes.")

# Agora, ao re-executar as outras células, elas usarão este valor mais preciso.


--- CALCULANDO P(TRACK) MÉDIO PONDERADO COM BASE NO INVENTÁRIO ---
O P(track) médio ponderado calculado é: 0.896
Este valor será usado em todos os cálculos subsequentes.


In [29]:
# ==============================================================================
# CÁLCULO DA CAPACIDADE DE DEFESA (SSPK IMPLÍCITO)
# ==============================================================================
print("\n--- INICIANDO OBJETIVO 1: CÁLCULO DA CAPACIDADE DE DEFESA ---")

df.rename(columns={'category': 'weapon_category'}, inplace=True)


N_SHOTS_MAPPING = {
    'ballistic missile': 2,
    'cruise missile': 2,
    'UAV': 1,
    'guided bomb': 1,
    'surface-to-air missile': 2,
    
}

def calculate_implied_sspk(launched, destroyed, p_track, n_shots):
    if launched == 0: return np.nan
    q = (launched - destroyed) / launched
    kw = 1 - q
    base_term = 1 - (kw / p_track)
    if base_term < 0: return 0.0
    sspk = 1 - (base_term)**(1 / n_shots)
    return sspk

# *** ANÁLISE ANUAL REINTRODUZIDA AQUI ***
stats_by_year = df.groupby(['year', 'weapon_category']).agg(
    launched=('launched', 'sum'),
    destroyed=('destroyed', 'sum')
).reset_index()

stats_by_year['n_shots'] = stats_by_year['weapon_category'].str.lower().map(N_SHOTS_MAPPING).fillna(1)
stats_by_year['implied_sspk'] = stats_by_year.apply(
    lambda row: calculate_implied_sspk(row['launched'], row['destroyed'], P_TRACK, row['n_shots']), axis=1
)
stats_by_year.to_excel("sspk_por_ano2.xlsx", index=False)
sspk_pivot_table = stats_by_year.pivot_table(
    index='weapon_category', columns='year', values='implied_sspk'
)
# Formata para percentual
sspk_pivot_table = sspk_pivot_table.applymap(lambda x: f"{x:.1%}" if pd.notnull(x) else "N/A") # type: ignore

print("\nEvolução Anual do SSPK Implícito (Capacidade de Interceptação):")
print(sspk_pivot_table)


# Análise Geral (agregada)
stats_by_category = df.groupby('weapon_category').agg(
    launched=('launched', 'sum'),
    destroyed=('destroyed', 'sum')
).reset_index()

stats_by_category['n_shots'] = stats_by_category['weapon_category'].str.lower().map(N_SHOTS_MAPPING).fillna(1)
stats_by_category['implied_sspk'] = stats_by_category.apply(
    lambda row: calculate_implied_sspk(row['launched'], row['destroyed'], P_TRACK, row['n_shots']), axis=1
)

print("\nSSPK Implícito por Categoria de Arma (Geral, todo o período):")
print(stats_by_category[stats_by_category['launched'] > 0].sort_values('implied_sspk', ascending=False).round(3))


--- INICIANDO OBJETIVO 1: CÁLCULO DA CAPACIDADE DE DEFESA ---

Evolução Anual do SSPK Implícito (Capacidade de Interceptação):
year                     2022   2023   2024   2025
weapon_category                                   
UAV                      0.0%  95.0%  78.4%  60.9%
ballistic missile         N/A  27.3%   7.9%  10.9%
cruise missile          55.1%  59.3%  50.2%  44.3%
guided bomb               N/A  89.3%  27.9%    N/A
surface-to-air missile   0.0%   0.0%   0.5%   0.0%

SSPK Implícito por Categoria de Arma (Geral, todo o período):
          weapon_category  launched  destroyed  n_shots  implied_sspk
3             guided bomb      14.0        9.0      1.0         0.717
0                     UAV   37355.0    23455.0      1.0         0.701
2          cruise missile    3719.0     2594.0      2.0         0.529
1       ballistic missile     500.0       84.0      2.0         0.099
4  surface-to-air missile     427.0        2.0      2.0         0.003


In [ ]:
# ==============================================================================
# TREINAMENTO DO MODELO DE ML SEM 
# ==============================================================================
print("\n--- OBJETIVO 2: Treinando o Modelo de Machine Learning")

# Preparação do DataFrame completo para o modelo
location_mapping = {'kyiv region': 'kyiv', 'kyivska': 'kyiv', 'kyiv and kyivska': 'kyiv', 'kyivska oblast': 'kyiv', 'kyivskyi': 'kyiv', 'kyiv city': 'kyiv', 'kharkiv region': 'kharkiv', 'kharkivska': 'kharkiv', 'odesa region': 'odesa', 'odeska': 'odesa', 'dnipropetrovsk region': 'dnipro', 'dnipropetrovska': 'dnipro', 'lviv region': 'lviv', 'lvivska': 'lviv', 'zaporizhzhia region': 'zaporizhzhia', 'mykolaiv region': 'mykolaiv', 'kherson region': 'kherson', 'poltava region': 'poltava', 'cherkasy region': 'cherkasy', 'khmelnytskyi region': 'khmelnytskyi', 'khmelnytska': 'khmelnytskyi', 'sumy region': 'sumy'}
df['main_target'] = df['target'].astype(str).str.split(',').str[0].str.strip().str.lower()
df['target_standardized'] = df['main_target'].replace(location_mapping)
model_df = df[df['launched'] > 0].copy()
model_df['interception_rate'] = model_df['destroyed'] / model_df['launched']
model_df['month'] = model_df['event_date'].dt.month

# Definindo as features (X) e o alvo (y)
target_col = 'interception_rate'
y = model_df[target_col]
# Removendo 'carrier' das features a serem usadas
X = model_df.drop(columns=[target_col, 'event_date', 'time_start', 'time_end', 'target', 'main_target', 'destroyed', 'carrier'])

# Divisão cronológica dos DataFrames
X_train_df = X[X['year'] < 2025]
X_test_df = X[X['year'] >= 2025]
y_train = y.loc[X_train_df.index]
y_test = y.loc[X_test_df.index]

# Identificar colunas numéricas e categóricas (sem 'carrier')
numeric_features = ['launched', 'year', 'month']
categorical_features = ['model', 'weapon_category', 'target_standardized'] 

# Lidar com valores nulos nas features categóricas para ambos os conjuntos
for col in categorical_features:
    X_train_df[col] = X_train_df[col].fillna('Unknown').astype(str)
    X_test_df[col] = X_test_df[col].fillna('Unknown').astype(str)

# Criar e treinar o encoder APENAS com os dados de treino
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder.fit(X_train_df[categorical_features])

# Aplicar o encoder treinado em ambos os conjuntos
X_train_encoded = encoder.transform(X_train_df[categorical_features])
X_test_encoded = encoder.transform(X_test_df[categorical_features])

# Combinar features numéricas e codificadas
X_train_final = np.hstack((X_train_df[numeric_features].values, X_train_encoded)) #type: ignore
X_test_final = np.hstack((X_test_df[numeric_features].values, X_test_encoded))#type: ignore

# --- Treinamento e Avaliação ---
if len(X_test_df) > 0 and len(X_train_df) > 0:
    model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    model.fit(X_train_final, y_train)
    predictions = model.predict(X_test_final)
    mae = mean_absolute_error(y_test, predictions)
    print(f"\nModelo Treinado. Erro Absoluto Médio (MAE) nos dados de 2025: {mae:.2%}")
else:
    model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    for col in categorical_features:
        X[col] = X[col].fillna('Unknown').astype(str)
    X_numeric_full = X[numeric_features].values
    X_encoded_full = encoder.fit_transform(X[categorical_features])
    X_final_full = np.hstack((X_numeric_full, X_encoded_full))
    model.fit(X_final_full, y)
    print("\nModelo Treinado com todos os dados disponíveis.")

# Salvando o novo modelo e encoder
joblib.dump(model, 'defense_model.joblib')
joblib.dump(encoder, 'model_encoder.joblib')
print("Novo modelo e encoder (sem 'carrier') salvos em arquivos .joblib.")


--- OBJETIVO 2: Treinando o Modelo de Machine Learning

Modelo Treinado (sem 'carrier'). Erro Absoluto Médio (MAE) nos dados de 2025: 15.17%
Novo modelo e encoder (sem 'carrier') salvos em arquivos .joblib.


In [31]:
# ==============================================================================
# PREDIÇÃO DE ATAQUE IMINENTE (CARREGANDO MODELO)
# ==============================================================================
print("\n--- INICIANDO PREDIÇÃO DE ATAQUE IMINENTE (CARREGANDO MODELO SALVO) ---")

try:
    loaded_model = joblib.load('defense_model.joblib')
    loaded_encoder = joblib.load('model_encoder.joblib')
    print("Modelo e encoder carregados dos arquivos com sucesso.")
except FileNotFoundError:
    print("ERRO: Arquivos 'defense_model.joblib' ou 'model_encoder.joblib' não encontrados.")
    print("Por favor, execute a Célula 4 para treinar e salvar o modelo primeiro.")
    # Interrompe a execução desta célula se os arquivos não existirem
    # exit() # Descomente se quiser que o script pare aqui em caso de erro

# Se o modelo foi carregado, prossiga com a predição
if 'loaded_model' in locals():
    
   
    imminent_attack_df = pd.read_csv('ataque_iminente.csv')
    print("Arquivo 'ataque_iminente.csv' carregado com sucesso.")


    prediction_df = pd.merge(imminent_attack_df, weapons_df[['model', 'category',]], on='model', how='left')
    prediction_df.rename(columns={'category': 'weapon_category'}, inplace=True)
    now = pd.to_datetime('now', utc=True).tz_convert('America/Sao_Paulo')
    prediction_df['year'] = now.year
    prediction_df['month'] = now.month
    prediction_df['main_target'] = prediction_df['target'].astype(str).str.strip().str.lower()
    prediction_df['target_standardized'] = prediction_df['main_target'].replace(location_mapping)
    categorical_features = ['model','weapon_category', 'target_standardized']
    for col in categorical_features:
        prediction_df[col] = prediction_df[col].fillna('Unknown').astype(str)

    # Aplicar o OneHotEncoder carregado
    X_pred_categorical = loaded_encoder.transform(prediction_df[categorical_features])
    X_pred_numeric = prediction_df[['launched', 'year', 'month']].values
    X_pred_final = np.hstack((X_pred_numeric, X_pred_categorical))

    # --- FAZER A PREDIÇÃO com o modelo carregado ---
    imminent_predictions = loaded_model.predict(X_pred_final)
    sanitized_predictions = np.clip(imminent_predictions, 0, 1)
    prediction_df['predicted_interception_rate'] = sanitized_predictions

    # --- Apresentar Resultados ---
    print("\n>>> Previsão de Resultados para o Ataque Iminente <<<")
    prediction_df['predicted_interception_rate_pct'] = prediction_df['predicted_interception_rate'].apply(lambda x: f"{x:.1%}")
    print(prediction_df[['model', 'launched', 'target', 'predicted_interception_rate_pct']])


--- INICIANDO PREDIÇÃO DE ATAQUE IMINENTE (CARREGANDO MODELO SALVO) ---
Modelo e encoder carregados dos arquivos com sucesso.
Arquivo 'ataque_iminente.csv' carregado com sucesso.

>>> Previsão de Resultados para o Ataque Iminente <<<
            model  launched target predicted_interception_rate_pct
0     X-101/X-555        16   lviv                           77.5%
1  Shahed-136/131        30  odesa                           86.0%
2      Iskander-M         4   kyiv                           72.1%
3      Iskander-M         2  odesa                           12.2%
4      Iskander-M         2   sumy                            0.0%
5  Shahed-136/131       352   kyiv                          100.0%
6      Iskander-M        11   kyiv                           63.2%
7      Iskander-K         5   kyiv                           86.3%


In [ ]:
# ==============================================================================
# CÉLULA 10: FUNÇÃO DE RETREINAMENTO DO MODELO COM NOVOS DADOS
# ==============================================================================
print("\n--- Definindo a Função de Retreinamento ---")

def retrain_model_with_new_data(new_attacks_df):
    """
    Retreina o modelo de machine learning com novos dados de ataque.

    Args:
        new_attacks_df (pd.DataFrame): Um DataFrame contendo os novos registros de ataques.
                                       Deve ter as mesmas colunas do arquivo original 'missile_attacks_daily.csv'.
    """
    print("Iniciando o processo de retreinamento...")
    
    # --- 1. Carregar Dados Antigos e Combinar com os Novos ---
    try:
        historical_attacks_df = pd.read_csv('missile_attacks_daily.csv')
        print(f"Carregados {len(historical_attacks_df)} registros históricos.")
    except FileNotFoundError:
        print("ERRO: Arquivo de dados históricos 'missile_attacks_daily.csv' não encontrado.")
        return False

    # Combinar dados históricos com os novos dados
    combined_attacks_df = pd.concat([historical_attacks_df, new_attacks_df], ignore_index=True)
    print(f"Total de registros após combinar com os {len(new_attacks_df)} novos: {len(combined_attacks_df)}")

    # --- 2. Re-executar todo o Pipeline de Processamento e Treinamento ---
    # Este código é uma réplica da Célula 5, agora dentro de uma função.
    
    # Merge com as informações das armas
    df_retrain = pd.merge(combined_attacks_df, weapons_df[['model', 'category']], on='model', how='left')
    df_retrain['event_date'] = pd.to_datetime(df_retrain['time_start'], format='mixed', errors='coerce')
    df_retrain.dropna(subset=['event_date'], inplace=True)
    df_retrain['year'] = df_retrain['event_date'].dt.year
    numeric_cols = ['launched', 'destroyed']
    for col in numeric_cols:
        df_retrain[col] = pd.to_numeric(df_retrain[col], errors='coerce').fillna(0)
    df_retrain['destroyed'] = df_retrain.apply(lambda row: min(row['launched'], row['destroyed']), axis=1)
    df_retrain['category'] = df_retrain['category'].fillna('Unknown')
    df_retrain.rename(columns={'category': 'weapon_category'}, inplace=True)
    
    # Padronização do alvo
    df_retrain['main_target'] = df_retrain['target'].astype(str).str.split(',').str[0].str.strip().str.lower()
    df_retrain['target_standardized'] = df_retrain['main_target'].replace(location_mapping)

    # Preparação final para o modelo
    model_df_retrain = df_retrain[df_retrain['launched'] > 0].copy()
    model_df_retrain['interception_rate'] = model_df_retrain['destroyed'] / model_df_retrain['launched']
    model_df_retrain['month'] = model_df_retrain['event_date'].dt.month
    
    target_col = 'interception_rate'
    y = model_df_retrain[target_col]
    X = model_df_retrain.drop(columns=[target_col, 'event_date', 'time_start', 'time_end', 'target', 'main_target', 'destroyed', 'carrier', 'guidance_system'])

    # Processamento de features
    numeric_features = ['launched', 'year', 'month']
    categorical_features = ['model', 'weapon_category', 'target_standardized']
    for col in categorical_features:
        X[col] = X[col].fillna('Unknown').astype(str)
        
    encoder_retrain = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    X_encoded_full = encoder_retrain.fit_transform(X[categorical_features])
    X_numeric_full = X[numeric_features].values
    X_final_full = np.hstack((X_numeric_full, X_encoded_full))

    # --- 3. Retreinamento do Modelo ---
    print("Retreinando o modelo com o conjunto de dados completo e atualizado...")
    model_retrain = GradientBoostingRegressor(n_estimators=100, random_state=42)
    model_retrain.fit(X_final_full, y)
    
    # --- 4. Salvando os Novos Artefatos ---
    joblib.dump(model_retrain, 'defense_model.joblib')
    joblib.dump(encoder_retrain, 'model_encoder.joblib')
    print("\nRETREINAMENTO COMPLETO!")
    print("Novas versões do 'defense_model.joblib' e 'model_encoder.joblib' foram salvas.")
    
    # Salvar o novo dataset completo para uso futuro
    combined_attacks_df.to_csv('missile_attacks_daily.csv', index=False)
    print("O arquivo 'missile_attacks_daily.csv' foi atualizado com os novos dados.")
    
    return True

# --- Exemplo de como usar a função ---
# Crie um DataFrame com seus novos dados. As colunas devem ser as mesmas do arquivo original.
# Este é apenas um exemplo.
new_data_example = {
    'time_start': ['2025-06-23 03:30'],
    'time_end': ['2025-06-23 04:15'],
    'model': ['Shahed-136/131'],
    'launch_place': ['Primorsko-Akhtarsk'],
    'target': ['mykolaiv'],
    'carrier': ['ground launcher'],
    'launched': [20],
    'destroyed': [18],
    'not_reach_goal': [1],
    'cross_border_belarus': [0],
    'back_russia': [0],
    'still_attacking': [0],
    'num_hit_location': [1],
    'num_fall_fragment_location': [3],
    'affected region': ['Mykolaivska'],
    'destroyed_details': ["18x UAVs 'Shahed'"],
    'launched_details': ["20x UAVs 'Shahed'"],
    'launch_place_details': ['Primorsko-Akhtarsk, Russia'],
    'source': ['Ukrainian Air Force']
}
new_attacks_example_df = pd.DataFrame(new_data_example)

# Para executar o retreinamento, descomente a linha abaixo:
# retrain_model_with_new_data(new_attacks_example_df)

In [32]:
# ==============================================================================
# CÉLULA 5 (AJUSTADA): CÁLCULO DE REQUISITOS PARA DEFESA 
# ==============================================================================
print("\n--- INICIANDO OBJETIVO 3: REQUISITOS PARA DEFESA ---")

# O SSPK para mísseis balísticos calculado anteriormente
ballistic_stats_row = stats_by_category[stats_by_category['weapon_category'] == 'ballistic missile']

if not ballistic_stats_row.empty:
    ballistic_stats = ballistic_stats_row.iloc[0]
    CURRENT_SSPK_BALLISTIC = calculate_implied_sspk(ballistic_stats['launched'], ballistic_stats['destroyed'], P_TRACK, n_shots=2)

    # *** AJUSTE PRINCIPAL: MUDANDO O OBJETIVO DE 99% PARA 90% ***
    TARGET_P0 = 0.10
    ATTACK_SIZE_W = 10
    
    print(f"Analisando um objetivo de {TARGET_P0:.0%} de probabilidade de que nenhum dos {ATTACK_SIZE_W} mísseis atinja o alvo.")

    def calculate_required_shots(target_p0, attack_size_W, p_track, sspk):
        if sspk >= 1.0: return 1
        # K_req é a probabilidade de abate necessária por alvo individual
        k_req = target_p0**(1/attack_size_W)
        
        # Verifica se o objetivo é matematicamente possível
        if k_req >= p_track:
            return float('inf') # Retorna infinito se K_req for maior que a capacidade de rastreamento
            
        log_term = 1 - (k_req / p_track)
        if log_term <= 0 or (1 - sspk) >= 1: return float('inf')
        
        n = np.log(log_term) / np.log(1 - sspk)
        return np.ceil(n)

    def calculate_required_sspk(target_p0, attack_size_W, p_track, n_shots):
        k_req = target_p0**(1/attack_size_W)
        if k_req >= p_track:
            return float('inf') # Retorna infinito se o objetivo for impossível
            
        base_term = 1 - (k_req / p_track)
        if base_term < 0: return float('inf')
        
        k = 1 - (1 - base_term)**(1/n_shots)
        return k

    # 1. Quantos disparos com o SSPK atual?
    req_n = calculate_required_shots(TARGET_P0, ATTACK_SIZE_W, P_TRACK, CURRENT_SSPK_BALLISTIC)
    print(f"\nCenário 1: Para atingir {TARGET_P0:.0%} de defesa com o SSPK atual de {CURRENT_SSPK_BALLISTIC:.1%}:")
    if np.isinf(req_n):
        print(f"   => Não é possível atingir o objetivo com o SSPK atual.")
    else:
        print(f"   => Seria necessário disparar {int(req_n)} interceptadores por míssil.")

    
else:
    print("\nNão foram encontrados dados para a categoria 'ballistic missile' para realizar a análise de requisitos.")

print("\n--- Análise de Cenário Ajustado Concluída ---")


--- INICIANDO OBJETIVO 3: REQUISITOS PARA DEFESA ---
Analisando um objetivo de 10% de probabilidade de que nenhum dos 10 mísseis atinja o alvo.

Cenário 1: Para atingir 10% de defesa com o SSPK atual de 9.9%:
   => Seria necessário disparar 21 interceptadores por míssil.

--- Análise de Cenário Ajustado Concluída ---
